[![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-181717?style=for-the-badge&logo=GitHub&link=https://github.com/Mearman/openalex-docs)](https://github.com/Mearman/openalex-docs)[![Open in GitHub](https://img.shields.io/badge/Open%20in-GitHub-181717?style=for-the-badge&logo=github&link=https://github.com/Mearman/openalex-docs/blob/main/api-entities/funders/search-funders.ipynb)](https://github.com/Mearman/openalex-docs/blob/main/api-entities/funders/search-funders.ipynb)[![Open in Colab](https://img.shields.io/badge/Open%20in-Colab-F9AB00?style=for-the-badge&logo=Google%20Colab&link=https://colab.research.google.com/github/Mearman/openalex-docs/blob/main/api-entities/funders/search-funders.ipynb)](https://colab.research.google.com/github/Mearman/openalex-docs/blob/main/api-entities/funders/search-funders.ipynb)

In [ ]:
%pip install --upgrade "git+https://github.com/Mearman/openalex-python-pydantic-v1.git"
%pip install pandasai

In [ ]:
import json
import pandas as pd
import numpy as np
from openalex_api import Configuration, ApiClient, AutocompleteApi, AuthorsApi, ConceptsApi, FundersApi, InstitutionsApi, PublishersApi, SourcesApi, WorksApi

configuration = Configuration(host="https://api.openalex.org")
autocomplete_api = AutocompleteApi(ApiClient(configuration))
authors_api = AuthorsApi(ApiClient(configuration))
concepts_api = ConceptsApi(ApiClient(configuration))
funders_api = FundersApi(ApiClient(configuration))
institutions_api = InstitutionsApi(ApiClient(configuration))
publishers_api = PublishersApi(ApiClient(configuration))
sources_api = SourcesApi(ApiClient(configuration))
works_api = WorksApi(ApiClient(configuration))

from pandasai import SmartDataframe
from pandasai.llm import OpenAI

In [ ]:
# @title  { run: "auto", display-mode: "form" }
openapi_token = "" # @param {type:"string"}

# Search funders

The best way to search for funders is to use the `search` query parameter, which searches the [`display_name`](./funder-object.md#display\_name), the [`alternate_titles`](./funder-object.md#alternate\_titles), and the [`description`](./funder_object.md#description) fields. Example:

* Search funders' `display_name`, `alternate_titles`, and `description` for "health":\
  [`https://api.openalex.org/funders?search=health`](https://api.openalex.org/funders?search=health)

In [ ]:
# @title { run: "auto", vertical-output: false }
# https://api.openalex.org/funders?search=health
search="health" # @param "health" {type: "string"}

response = funders_api.get_funders(
	search=search
)

df = pd.DataFrame(response.results)
display(df)

In [ ]:
numeric_df = df[['id', 'display_name'] +
	[col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col != 'relevance_score']]
display(numeric_df)
try:
	llm = OpenAI(api_token = openapi_token)
	sdf = SmartDataframe(numeric_df, config = { "llm": llm })
	sdf.chat("Plot a chart of this data")
except:
	print("Error when creating SmartDataframe")

{% hint style="info" %}
You can read more about search [here](./../../how-to-use-the-api/get-lists-of-entities/search-entities.ipynb). It will show you how relevance score is calculated, how words are stemmed to improve search results, and how to do complex boolean searches.
{% endhint %}

## Search a specific field

You can also use search as a [filter](./../../how-to-use-the-api/get-lists-of-entities/filter-entity-lists.ipynb), allowing you to fine-tune the fields you're searching over. To do this, you append `.search` to the end of the property you are filtering for:

* Get funders with "florida" in the `display_name`:\
  [`https://api.openalex.org/funders?filter=display_name.search:florida`](https://api.openalex.org/funders?filter=display_name.search:florida)

In [ ]:
# @title { run: "auto", vertical-output: false }
# https://api.openalex.org/funders?filter=display_name.search:florida
filter="display_name.search:florida" # @param "display_name.search:florida" {type: "string"}

response = funders_api.get_funders(
	filter=filter
)

df = pd.DataFrame(response.results)
display(df)

In [ ]:
numeric_df = df[['id', 'display_name'] +
	[col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col != 'relevance_score']]
display(numeric_df)
try:
	llm = OpenAI(api_token = openapi_token)
	sdf = SmartDataframe(numeric_df, config = { "llm": llm })
	sdf.chat("Plot a chart of this data")
except:
	print("Error when creating SmartDataframe")

The following fields can be searched as a filter within funders:

| Search filter                                                        | Field that is searched                                |
| -------------------------------------------------------------------- | ----------------------------------------------------- |
| [`display_name.search`](./filter-funders.md#display_name.search) | [`display_name`](./funder-object.md#display_name) |
| [`description.search`](./filter-funders.md#description.search) | [`description`](./funder-object.md#description) |

You can also use the filter `default.search`, which works the same as using the [`search` parameter](./README.md#search-funders).

## Autocomplete funders

You can autocomplete funders to create a very fast type-ahead style search function:

* Autocomplete funders with "national sci" in the [`display_name`](./funder-object.md#display_name):\
  [https://api.openalex.org/autocomplete/funders?q=national+sci](https://api.openalex.org/autocomplete/funders?q=national+sci)

In [ ]:
# @title { run: "auto", vertical-output: false }
# https://api.openalex.org/autocomplete/funders?q=national+sci
q="national sci" # @param "national sci" {type: "string"}

response = autocomplete_api.get_autocomplete_funders(
	q=q
)

df = pd.DataFrame(response.results)
display(df)

In [ ]:
numeric_df = df[['id', 'display_name'] +
	[col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col != 'relevance_score']]
display(numeric_df)
try:
	llm = OpenAI(api_token = openapi_token)
	sdf = SmartDataframe(numeric_df, config = { "llm": llm })
	sdf.chat("Plot a chart of this data")
except:
	print("Error when creating SmartDataframe")

This returns a list of funders with the funder location set as the hint:

```json
  "results": [
    {
        "id": "https://openalex.org/F4320306076",
        "display_name": "National Science Foundation",
        "hint": null,
        "cited_by_count": 6705777,
        "works_count": 264303,
        "entity_type": "funder",
        "external_id": "https://ror.org/021nxhr62"
    },
    ...
  ]
```

{% hint style="info" %}
Read more in the [autocomplete page](./../../how-to-use-the-api/get-lists-of-entities/autocomplete-entities.ipynb) in the API guide.
{% endhint %}